In [43]:
import os, shutil
import time
import matplotlib.pyplot as plt
import scipy
import numpy as np
from PIL import Image
from scipy import ndimage
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras import models
from keras import layers

In [18]:
train_folder = 'chest_xray/train/'
test_folder = 'chest_xray/test/'
val_folder = 'chest_xray/val/'

In [34]:
# get all the data in the directory split/test (180 images), and reshape them
test_generator = ImageDataGenerator(rescale=1./255).flow_from_directory(
        test_folder, batch_size=624) 

# get all the data in the directory split/validation (200 images), and reshape them
val_generator = ImageDataGenerator(rescale=1./255).flow_from_directory(
        val_folder, batch_size=16)

# get all the data in the directory split/train (542 images), and reshape them
train_generator = ImageDataGenerator(rescale=1./255).flow_from_directory(
        train_folder, batch_size=5216)

Found 624 images belonging to 2 classes.
Found 16 images belonging to 2 classes.
Found 5216 images belonging to 2 classes.


In [35]:
# create the data sets
train_images, train_labels = next(train_generator)
test_images, test_labels = next(test_generator)
val_images, val_labels = next(val_generator)

In [37]:
# Explore your dataset again
m_train = train_images.shape[0]
num_px = train_images.shape[1]
m_test = test_images.shape[0]
m_val = val_images.shape[0]

print ("Number of training samples: " + str(m_train))
print ("Number of testing samples: " + str(m_test))
print ("Number of validation samples: " + str(m_val))
print ("train_images shape: " + str(train_images.shape))
print ("train_labels shape: " + str(train_labels.shape))
print ("test_images shape: " + str(test_images.shape))
print ("test_labels shape: " + str(test_labels.shape))
print ("val_images shape: " + str(val_images.shape))
print ("val_labels shape: " + str(val_labels.shape))

Number of training samples: 5216
Number of testing samples: 624
Number of validation samples: 16
train_images shape: (5216, 256, 256, 3)
train_labels shape: (5216, 2)
test_images shape: (624, 256, 256, 3)
test_labels shape: (624, 2)
val_images shape: (16, 256, 256, 3)
val_labels shape: (16, 2)


In [38]:
train_img = train_images.reshape(train_images.shape[0], -1)
test_img = test_images.reshape(test_images.shape[0], -1)
val_img = val_images.reshape(val_images.shape[0], -1)

print(train_img.shape)
print(test_img.shape)
print(val_img.shape)

(5216, 196608)
(624, 196608)
(16, 196608)


In [45]:
train_y = np.reshape(train_labels[:,0], (5216,1))
test_y = np.reshape(test_labels[:,0], (624,1))
val_y = np.reshape(val_labels[:,0], (16,1))

In [46]:
model = models.Sequential()
model.add(layers.Dense(20, activation='relu', input_shape=(196608,))) # 2 hidden layers
model.add(layers.Dense(7, activation='relu'))
model.add(layers.Dense(5, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

In [47]:
model.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

history = model.fit(train_img,
                    train_y,
                    epochs=50,
                    batch_size=32,
                    validation_data=(val_img, val_y))

Epoch 1/50
163/163 [==============================] - 6s 35ms/step - loss: 0.5258 - accuracy: 0.7458 - val_loss: 0.6886 - val_accuracy: 0.6875
Epoch 2/50
163/163 [==============================] - 5s 29ms/step - loss: 0.3648 - accuracy: 0.8530 - val_loss: 0.7393 - val_accuracy: 0.6875
Epoch 3/50
163/163 [==============================] - 5s 28ms/step - loss: 0.3078 - accuracy: 0.8949 - val_loss: 0.8179 - val_accuracy: 0.6875
Epoch 4/50
163/163 [==============================] - 4s 27ms/step - loss: 0.2805 - accuracy: 0.9061 - val_loss: 1.0469 - val_accuracy: 0.6250
Epoch 5/50
163/163 [==============================] - 4s 27ms/step - loss: 0.2550 - accuracy: 0.9185 - val_loss: 0.4567 - val_accuracy: 0.8750
Epoch 6/50
163/163 [==============================] - 5s 28ms/step - loss: 0.2380 - accuracy: 0.9258 - val_loss: 0.5670 - val_accuracy: 0.7500
Epoch 7/50
163/163 [==============================] - 4s 26ms/step - loss: 0.2385 - accuracy: 0.9248 - val_loss: 0.6491 - val_accuracy: 0.7500

In [48]:
results_train = model.evaluate(train_img, train_y)

163/163 [==============================] - 2s 12ms/step - loss: 0.0881 - accuracy: 0.9720


In [49]:
results_val = model.evaluate(val_img, val_y)

1/1 [==============================] - 0s 2ms/step - loss: 0.3877 - accuracy: 0.8750
